<p align="center">
  <img src="https://user-images.githubusercontent.com/90031508/183531098-494a5819-7714-4f72-8ff8-d038982eb5f0.png" alt="Water Oracle logo"/>
</p>



This Work is adapted from 'Tensorflow example workflows', 
https://developers.google.com/earth-engine/guides/tf_examples examples.
Copyright 2020 Google LLC. https://www.apache.org/licenses/LICENSE-2.0

Please run this notebook on google colab (pro+)

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/drive/1vQd4gM12G3LD51H-7qSUgtgs04dX3xCF?usp=sharing">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/ese-msc-2021/irp-kl121"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Introduction

## Prerequisites
- Google account and logins
- Google colab subscription with pro or pro+ is optional but would help with long runtime
- Google cloud platform account in order to use google cloud bucket. (Note that you would need sufficient funds to store large amount of models and training data.)
- Wandb.ai account which is free of charge


## What is this notebook?


The purpose of this notebook is to train the models that has been stored in the google cloud buckets by the `Preprocessing_and_export_global.ipynb` notebook. The training is accompanied with Wandb and is splitted into three different methods of training

- Feature stack deep learning with UNET
- Multiview learning with 2 inputs and modified UNET
- Multiview learning with 3 inputs and modified UNET

## Creating Packages

Creating the tools packages that will be used throughout the notebook. The package includes 
- metrics_.py
- config.py
- preprocessing.py
- model.py

In [ ]:
PACKAGE_PATH = 'tools'

!ls -l
!mkdir {PACKAGE_PATH}
!touch {PACKAGE_PATH}/__init__.py
!ls -l {PACKAGE_PATH}

total 4
drwxr-xr-x 1 root root 4096 Aug 15 13:44 sample_data
total 0
-rw-r--r-- 1 root root 0 Aug 24 18:29 __init__.py


In [ ]:
%%writefile {PACKAGE_PATH}/metrics_.py

from keras import backend as K
import tqdm.notebook as tq
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

CONFIG = None

__all__ = ["f1", "custom_accuracy", "MetricCalculator",
           "MetricCalculator_multiview_2", "MetricCalculator_multiview_3",
           "MetricCalculator_NDWI", "ndwi_threashold"]


def f1(y_true, y_pred):
    """
    The function is used as tensorflow metrics when training.
    It takes in the ground truth and the model predicted result
    and evaluate the F1 score. This is an experimental function
    and should not be used as further model training metric.

    Parameters
    ----------
    y_true : tf.tensor
    y_pred : tf.tensor

    Returns
    ----------
    F1 score in keras backend

    Notes
    -----
    This function is flawed because keras calculates the metrics batchwise
    which is why F1 metric is removed from keras. To properly calulate the
    F1 score, we can use the callback function or manually calculate F1
    score after the model has finished training. The latter is chosen
    and this could be seen in MetricCalculator, MetricCalculator_multiview_2
    and MetricCalculator_multiview_3.

    The reason this function is kept is because the model was initially
    trained with these metrics and stored in the google cloud bucket.
    To retrieve the models these metrics must be passed inorder
    to retrieve the model. Since the model is optimize on the loss rather
    than the metrics, the incorrect metric would not effect the model
    training process. The code is obtained/modified from:

    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

    https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
    """
    def recall(y_true, y_pred):
        """
        Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """
        Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


def custom_accuracy(y_true, y_pred):
    """
    The function is used as tensorflow metrics when training.
    It takes in the ground truth and the model predicted result
    and evaluate the accuracy score. This is an experimental function
    and should not be used as further model training metric.

    Parameters
    ----------
    y_true : tf.tensor
    y_pred : tf.tensor

    Returns
    ----------
    accuracy score in keras backend

    Notes
    -----
    This function is modified from the F1 metric above to fit
    the definition of accuracy. However, tensorflow's
    "categorical_accuracy" is used instead. The accuracy metric
    would also be recalculated again in MetricCalculator,
    MetricCalculator_multiview_2 and MetricCalculator_multiview_3.

    The reason this function is kept is because the model was
    initially trained with these metrics and stored in
    the google cloud bucket. To retrieve the models these
    metrics must be passed inorder to retrieve the model.
    Since the model is optimize on the loss rather than
    the metrics, the incorrect metric would not effect
    the model training process. The code is obtained/modified from:

    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

    https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
    """
    # total_data = K.int_shape(y_true) + K.int_shape(y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    true_negatives = K.sum(K.round(K.clip(1 - y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    total_data = - true_positives + true_negatives + \
        possible_positives + predicted_positives
    return (true_positives + true_negatives) / (total_data + K.epsilon())


def MetricCalculator(model, test_data, total_steps):
    """
    This function takes in the feature stack model loaded
    from google cloud bucket, the test_data which is the
    tensor object and the number of steps and returns
    the metrics including accuracy, recall, precision and F1

    Parameters
    ----------
    model : keras.engine.functional.Functional
    test_data : RepeatDataset with tf.float32
    total_steps : int/float

    Returns
    ----------
    Returns the precision, recall, f1, accuracy
    metric based on the model performance.

    Notes
    -----
    This function should be used instead of the F1, custom_accuracy
    written above. The code is obtained/modified from:

    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

    https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
    """
    pred = []
    true = []
    pbar = tq.tqdm(total=total_steps)
    for steps, data in enumerate(test_data):
        pbar.update(1)
        if steps == total_steps:
            break
        input = data[0]
        y_true = data[1]
        y_pred = np.rint(model.predict(input))
        y_true = np.reshape(y_true, (256 * 256, 2))
        y_pred = np.reshape(y_pred, (256 * 256, 2))
        pred.append(y_pred)
        true.append(y_true)
    f1_macro = f1_score(np.reshape(true, (total_steps * 65536, 2)),
                        np.reshape(pred, (total_steps * 65536, 2)),
                        average="macro")
    recall_macro = recall_score(np.reshape(true, (total_steps * 65536, 2)),
                                np.reshape(pred, (total_steps * 65536, 2)),
                                average="macro")
    precision_macro = precision_score(np.reshape(true,
                                      (total_steps * 65536, 2)),
                                      np.reshape(pred,
                                      (total_steps * 65536, 2)),
                                      average="macro")
    accuracy = accuracy_score(np.reshape(true, (total_steps * 65536, 2)),
                              np.reshape(pred, (total_steps * 65536, 2)))

    print("precision_macro: ", precision_macro)
    print("recall_macro: ", recall_macro)
    print("F1_macro_Score: : ", f1_macro)
    print("Accuracy: ", accuracy)

    return precision_macro, recall_macro, f1_macro, accuracy


def MetricCalculator_multiview_2(model, test_data, total_steps):
    """
    This function takes in the multiview-2 model loaded
    from google cloud bucket, the test_data which is the
    tensor object and the number of steps and returns
    the metrics including accuracy, recall, precision and F1

    Parameters
    ----------
    model : keras.engine.functional.Functional
    test_data : RepeatDataset with tf.float32
    total_steps : int/float

    Returns
    ----------
    Returns the precision, recall, f1, accuracy metric
    based on the model performance.

    Notes
    -----
    This function should be used instead of the F1,
    custom_accuracy written above. The code is obtained/modified from:

    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

    https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
    """
    pbar = tq.tqdm(total=total_steps)
    pred = []
    true = []
    for steps, data in enumerate(test_data):
        pbar.update(1)
        if steps >= total_steps:
            break
        input = data[0]
        x1, x2 = tf.split(input, [len(CONFIG.BANDS1), len(CONFIG.BANDS2)], 3)
        y_true = data[1]
        y_pred = np.rint(model.predict([x1, x2]))
        y_true = np.reshape(y_true, (256 * 256, 2))
        y_pred = np.reshape(y_pred, (256 * 256, 2))
        pred.append(y_pred)
        true.append(y_true)
    f1_macro = f1_score(np.reshape(true, (total_steps * 65536, 2)),
                        np.reshape(pred, (total_steps * 65536, 2)),
                        average="macro")
    recall_macro = recall_score(np.reshape(true, (total_steps * 65536, 2)),
                                np.reshape(pred, (total_steps * 65536, 2)),
                                average="macro")
    precision_macro = precision_score(np.reshape(true,
                                      (total_steps * 65536, 2)),
                                      np.reshape(pred,
                                      (total_steps * 65536, 2)),
                                      average="macro")
    accuracy = accuracy_score(np.reshape(true, (total_steps * 65536, 2)),
                              np.reshape(pred, (total_steps * 65536, 2)))

    print("precision_macro: ", precision_macro)
    print("recall_macro: ", recall_macro)
    print("F1_macro_Score: : ", f1_macro)
    print("Accuracy: ", accuracy)

    return precision_macro, recall_macro, f1_macro, accuracy


def MetricCalculator_multiview_3(model, test_data, total_steps):
    """
    This function takes in the multiview-3 model loaded from
    google cloud bucket, the test_data which is the tensor object
    and the number of steps and returns the metrics including
    accuracy, recall, precision and F1

    Parameters
    ----------
    model : keras.engine.functional.Functional
    test_data : RepeatDataset with tf.float32
    total_steps : int/float

    Returns
    ----------
    Returns the precision, recall, f1,
    accuracy metric based on the model performance.

    Notes
    -----
    This function should be used instead of the F1,
    custom_accuracy written above. The code is obtained/modified from:

    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

    https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
    """
    pbar = tq.tqdm(total=total_steps)
    pred = []
    true = []
    for steps, data in enumerate(test_data):
        pbar.update(1)
        if steps >= total_steps:
            break
        input = data[0]
        x1, x2, x3 = tf.split(input,
                              [len(CONFIG.BANDS1),
                               len(CONFIG.BANDS2),
                               len(CONFIG.BANDS3)],
                              3)
        y_true = data[1]
        y_pred = np.rint(model.predict([x1, x2, x3]))
        y_true = np.reshape(y_true, (256 * 256, 2))
        y_pred = np.reshape(y_pred, (256 * 256, 2))
        pred.append(y_pred)
        true.append(y_true)
    f1_macro = f1_score(np.reshape(true, (total_steps * 65536, 2)),
                        np.reshape(pred, (total_steps * 65536, 2)),
                        average="macro")
    recall_macro = recall_score(np.reshape(true, (total_steps * 65536, 2)),
                                np.reshape(pred, (total_steps * 65536, 2)),
                                average="macro")
    precision_macro = precision_score(np.reshape(true,
                                      (total_steps * 65536, 2)),
                                      np.reshape(pred,
                                      (total_steps * 65536, 2)),
                                      average="macro")
    accuracy = accuracy_score(np.reshape(true, (total_steps * 65536, 2)),
                              np.reshape(pred, (total_steps * 65536, 2)))

    print("precision_macro: ", precision_macro)
    print("recall_macro: ", recall_macro)
    print("F1_macro_Score: : ", f1_macro)
    print("Accuracy: ", accuracy)

    return precision_macro, recall_macro, f1_macro, accuracy


def ndwi_threashold(B3, B5):
    """
    This function takes in bands 3 and bands 5 from the landsat
    imagery and returns the tuple prediction of whether
    there is water present or not. The threashold is set at 0.

    Parameters
    ----------
    test_data : RepeatDataset with tf.float32
    total_steps : int/float

    Returns
    ----------
    tuple of whether there is water or not
    """
    ndwi = (B3 - B5) / (B3 + B5)
    if ndwi > 0:
        return 0, 1
    else:
        return 1, 0


def MetricCalculator_NDWI(test_data, total_steps):
    """
    This function takes in the test_data which is the tensor object and
    the number of steps and returns the metrics including accuracy,
    recall, precision and F1 for NDWI performance.

    Parameters
    ----------
    test_data : RepeatDataset with tf.float32
    total_steps : int/float

    Returns
    ----------
    Returns the precision, recall, f1, accuracy metric
    based on the NDWI performance
    """
    pred = []
    true = []
    pbar = tq.tqdm(total=total_steps)
    for steps, data in enumerate(test_data):
        # print(f'Number of steps: {steps}', end = "\r")
        pbar.update(1)
        if steps == total_steps:
            break
        input = data[0]
        y_true = data[1]
        input = np.reshape(input, (256 * 256, 2))
        y_pred = []
        for i in range(256 * 256):
            B3, B5 = input[i]
            first, second = ndwi_threashold(B3, B5)
            y_pred.append([first, second])
        y_true = np.reshape(y_true, (256 * 256, 2))
        y_pred = np.reshape(y_pred, (256 * 256, 2))
        pred.append(y_pred)
        true.append(y_true)
    f1_macro = f1_score(np.reshape(true, (total_steps * 65536, 2)),
                        np.reshape(pred, (total_steps * 65536, 2)),
                        average="macro")
    recall_macro = recall_score(np.reshape(true, (total_steps * 65536, 2)),
                                np.reshape(pred, (total_steps * 65536, 2)),
                                average="macro")
    precision_macro = precision_score(np.reshape(true,
                                                 (total_steps * 65536, 2)),
                                      np.reshape(pred,
                                                 (total_steps * 65536, 2)),
                                      average="macro")
    accuracy = accuracy_score(np.reshape(true, (total_steps * 65536, 2)),
                              np.reshape(pred, (total_steps * 65536, 2)))

    print("precision_macro: ", precision_macro)
    print("recall_macro: ", recall_macro)
    print("F1_macro_Score: : ", f1_macro)
    print("Accuracy: ", accuracy)

    return precision_macro, recall_macro, f1_macro, accuracy


Writing tools/metrics_.py


In [ ]:
%%writefile {PACKAGE_PATH}/model.py

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers

CONFIG = None

__all__ = ["conv_block", "EncoderMiniBlock", "DecoderMiniBlock",
           "CustomModel", "get_model", "CustomModel_multiview_2",
           "get_model_multiview_2", "CustomModel_multiview_3",
           "get_model_multiview_3", "get_model_multiview_2_HT"]


def conv_block(input_tensor, num_filters):
    """
    This is processes the tensor right after the encoder
    to give the center block. The function takes in input tensor
    and number of filters and returns the next layer which is the
    center layer.

    Parameters
    ----------
    input_tensor : tf.float32/tf.int
    num_filters : int/float

    Returns
    ----------
    returns the next layer which is the center layer which is a tensor object

    Notes
    -----
    The code is obtained/modified from:

    https://medium.com/geekculture/u-net-implementation-from-scratch-using-tensorflow-b4342266e406

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
    encoder = layers.BatchNormalization()(encoder)
    encoder = layers.Activation('relu')(encoder)
    encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
    encoder = layers.BatchNormalization()(encoder)
    encoder = layers.Activation('relu')(encoder)
    return encoder


def EncoderMiniBlock(inputs, num_filters=32,
                     dropout_prob=0.3, max_pooling=True):
    """
    Encoder miniblock that will enable creation of all other encoder layers in
    the get_model function. The function takes in inputs, number of filter,
    a dropout probability and max_pooling parameter. The function
    returns the next layer and the corresponding layer which will
    be used in decoding later on.

    Parameters
    ----------
    input_tensor : tf.float32/tf.int
    num_filters : int/float
    dropout_prob : float
    max_pooling : bool

    Returns
    ----------
    The function returns the next layer and the corresponding layer which
    will be used in decoding later on as a tensor object

    Notes
    -----
    The code is obtained/modified from:

    https://medium.com/geekculture/u-net-implementation-from-scratch-using-tensorflow-b4342266e406

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    conv = layers.Conv2D(num_filters,
                         3,  # filter size
                         activation='relu',
                         padding='same',
                         kernel_initializer='HeNormal')(inputs)
    conv = layers.Conv2D(num_filters,
                         3,  # filter size
                         activation='relu',
                         padding='same',
                         kernel_initializer='HeNormal')(conv)

    conv = layers.BatchNormalization()(conv, training=False)
    if dropout_prob > 0:
        conv = tf.keras.layers.Dropout(dropout_prob)(conv)
    if max_pooling:
        next_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv)
    else:
        next_layer = conv
    skip_connection = conv
    return next_layer, skip_connection


def DecoderMiniBlock(prev_layer_input, skip_layer_input, num_filters=32):
    """
    Decoder miniblock will enable creation of all other
    decoder layers in the get_model function.
    The function takes in the previous layer inputs,
    the corresponding encoder and number of filters.
    The function returns the next layer and the corresponding
    layer which will be used in decoding later on.

    Parameters
    ----------
    prev_layer_input : tf.float32/tf.int
    skip_layer_input : tf.float32/tf.int
    num_filters : int/float

    Returns
    ----------
    The function returns the next layer and the corresponding
    layer which will be used in decoding later on as a tensor object

    Notes
    -----
    The code is obtained/modified from:

    https://medium.com/geekculture/u-net-implementation-from-scratch-using-tensorflow-b4342266e406

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    up = layers.Conv2DTranspose(
        num_filters,
        (3, 3),
        strides=(2, 2),
        padding='same')(prev_layer_input)
    merge = layers.concatenate([up, skip_layer_input], axis=3)
    conv = layers.Conv2D(num_filters,
                         3,
                         activation='relu',
                         padding='same',
                         kernel_initializer='HeNormal')(merge)
    conv = layers.Conv2D(num_filters,
                         3,
                         activation='relu',
                         padding='same',
                         kernel_initializer='HeNormal')(conv)
    return conv


class CustomModel(tf.keras.Model):
    """
    This class allows us to create custom model by modifying
    the functions of interest including the train_step test_step
    in order to enable the model to take in multilayered inputs.
    Also, the execution is switched from
    eager to graph in order to increase the speed of training

    Notes
    -----
    The code is obtained/modified from:

    https://towardsdatascience.com/eager-execution-vs-graph-execution-which-is-better-38162ea4dbf6#:~:text=Eager%20execution%20is%20a%20powerful,they%20occur%20in%20your%20code.

    https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit
    """
    @tf.function
    def train_step(self, data):
        """
        This function is a standard train_step in tensorflow,
        but graph execution is used instead. The function
        takes in the data and return the corresponding metrics

        Parameters
        ----------
        data : tuple of tf.float32/tf.int

        Returns
        ----------
        The function returns the corresponding metrics
        """
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y,
                                      y_pred,
                                      regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    @tf.function
    def test_step(self, data):
        """
        This function is a standard test_step in tensorflow,
        but graph execution is used instead.
        The function takes in the data and
        return the corresponding metrics

        Parameters
        ----------
        data : tuple of tf.float32/tf.int

        Returns
        ----------
        The function returns the corresponding metrics
        """
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False)
        # Updates the metrics tracking the loss
        self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        # Update the metrics.
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}


def get_model():
    """
    This function puts all the previous mini encoders,
    decoder and conv_block and the modified custom model
    together in order to compile and return a customized
    model for feature stack method.

    Notes
    -----
    The code is obtained/modified from:

    https://towardsdatascience.com/eager-execution-vs-graph-execution-which-is-better-38162ea4dbf6#:~:text=Eager%20execution%20is%20a%20powerful,they%20occur%20in%20your%20code.

    https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit
    """
    inputs = layers.Input(shape=[None, None, len(CONFIG.BANDS)])  # 256
    encoder0_pool, encoder0 = EncoderMiniBlock(inputs, 32)  # 128
    encoder1_pool, encoder1 = EncoderMiniBlock(encoder0_pool, 64)  # 64
    encoder2_pool, encoder2 = EncoderMiniBlock(encoder1_pool, 128)  # 32
    encoder3_pool, encoder3 = EncoderMiniBlock(encoder2_pool, 256)  # 16
    encoder4_pool, encoder4 = EncoderMiniBlock(encoder3_pool, 512)  # 8
    center = conv_block(encoder4_pool, 1024)  # center
    decoder4 = DecoderMiniBlock(center, encoder4, 512)  # 16
    decoder3 = DecoderMiniBlock(decoder4, encoder3, 256)  # 32
    decoder2 = DecoderMiniBlock(decoder3, encoder2, 128)  # 64
    decoder1 = DecoderMiniBlock(decoder2, encoder1, 64)  # 128
    decoder0 = DecoderMiniBlock(decoder1, encoder0, 32)  # 256
    outputs = layers.Dense(2, activation=tf.nn.softmax)(decoder0)

    model_custom = CustomModel(inputs, outputs)

    model_custom.compile(
        optimizer=optimizers.get(CONFIG.OPTIMIZER),
        loss=losses.get(CONFIG.LOSS),
        metrics=[CONFIG.METRICS]
    )
    return model_custom


class CustomModel_multiview_2(tf.keras.Model):
    """
    This class allows us to create custom model by
    modifying the functions of interest including the train_step
    test_step in order to enable the model to take in 2 layer
    inputs for multiview learning. Also, the execution is switched from
    eager to graph in order to increase the speed of training

    Notes
    -----
    The code is obtained/modified from:

    https://towardsdatascience.com/eager-execution-vs-graph-execution-which-is-better-38162ea4dbf6#:~:text=Eager%20execution%20is%20a%20powerful,they%20occur%20in%20your%20code.

    https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit
    """
    @tf.function
    def train_step(self, data):
        """
        This function modifies the standard train_step in
        tensorflow in order to manipulate and split the
        input data to put into the multiview deep learning model,
        and graph execution is used instead.
        The function takes in the data and return the corresponding
        metrics.

        Parameters
        ----------
        data : tuple of tf.float32/tf.int

        Returns
        ----------
        The function returns the corresponding metrics
        """
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data
        x1, x2 = tf.split(x, [len(CONFIG.BANDS1), len(CONFIG.BANDS2)], 3)
        # print(x.numpy())

        with tf.GradientTape() as tape:
            y_pred = self([x1, x2], training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y,
                                      y_pred,
                                      regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    @tf.function
    def test_step(self, data):
        """
        This function modifies the standard test_step in tensorflow
        in order to manipulate and split the input data to put into
        the multiview deep learning model, and graph execution is used instead.
        The function takes in the data and return the corresponding metrics

        Parameters
        ----------
        data : tuple of tf.float32/tf.int

        Returns
        ----------
        The function returns the corresponding metrics
        """
        # Unpack the data
        x, y = data
        x1, x2 = tf.split(x, [len(CONFIG.BANDS1), len(CONFIG.BANDS2)], 3)
        # Compute predictions
        y_pred = self([x1, x2], training=False)
        # Updates the metrics tracking the loss
        self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        # Update the metrics.
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}


def get_model_multiview_2():
    """
    This function puts all the previous mini encoders,
    decoder and conv_block and the modified custom model
    together in order to compile and return a customized
    model for multiview learning with 2 inputs

    Notes
    -----
    The code is obtained/modified from:

    https://towardsdatascience.com/eager-execution-vs-graph-execution-which-is-better-38162ea4dbf6#:~:text=Eager%20execution%20is%20a%20powerful,they%20occur%20in%20your%20code.

    https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit
    """
    # First input
    first_input = layers.Input(shape=[None, None, len(CONFIG.BANDS1)])  # 256
    # Encoding section
    first_encoder0_pool, first_encoder0 = \
        EncoderMiniBlock(first_input, 32)  # 128
    first_encoder1_pool, first_encoder1 = \
        EncoderMiniBlock(first_encoder0_pool, 64)  # 64
    first_encoder2_pool, first_encoder2 = \
        EncoderMiniBlock(first_encoder1_pool, 128)  # 32
    first_encoder3_pool, first_encoder3 = \
        EncoderMiniBlock(first_encoder2_pool, 256)  # 16
    first_encoder4_pool, first_encoder4 = \
        EncoderMiniBlock(first_encoder3_pool, 512)  # 8
    # Center block
    first_center = conv_block(first_encoder4_pool, 1024)
    # Decoding
    first_decoder4 = \
        DecoderMiniBlock(first_center, first_encoder4, 512)  # 16
    first_decoder3 = \
        DecoderMiniBlock(first_decoder4, first_encoder3, 256)  # 32
    first_decoder2 = \
        DecoderMiniBlock(first_decoder3, first_encoder2, 128)  # 64
    first_decoder1 = \
        DecoderMiniBlock(first_decoder2, first_encoder1, 64)  # 128
    first_decoder0 = \
        DecoderMiniBlock(first_decoder1, first_encoder0, 32)  # 256

    # Second input
    second_input = layers.Input(shape=[None, None, len(CONFIG.BANDS2)])  # 256
    # Encoding section
    second_encoder0_pool, second_encoder0 = \
        EncoderMiniBlock(second_input, 32)  # 128
    second_encoder1_pool, second_encoder1 = \
        EncoderMiniBlock(second_encoder0_pool, 64)  # 64
    second_encoder2_pool, second_encoder2 = \
        EncoderMiniBlock(second_encoder1_pool, 128)  # 32
    second_encoder3_pool, second_encoder3 = \
        EncoderMiniBlock(second_encoder2_pool, 256)  # 16
    second_encoder4_pool, second_encoder4 = \
        EncoderMiniBlock(second_encoder3_pool, 512)  # 8
    # Center block
    second_center = conv_block(second_encoder4_pool, 1024)  # center
    # Decoder section
    second_decoder4 = \
        DecoderMiniBlock(second_center, second_encoder4, 512)  # 16
    second_decoder3 = \
        DecoderMiniBlock(second_decoder4, second_encoder3, 256)  # 32
    second_decoder2 = \
        DecoderMiniBlock(second_decoder3, second_encoder2, 128)  # 64
    second_decoder1 = \
        DecoderMiniBlock(second_decoder2, second_encoder1, 64)  # 128
    second_decoder0 = \
        DecoderMiniBlock(second_decoder1, second_encoder0, 32)  # 256

    # Fuse two features
    concat_output = tf.keras.layers.concatenate([first_decoder0,
                                                 second_decoder0],
                                                name='cca_output')
    outputs = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(concat_output)

    model_custom = CustomModel_multiview_2([first_input,
                                            second_input],
                                           outputs)
    model_custom.compile(
        optimizer=optimizers.get(CONFIG.OPTIMIZER),
        loss=losses.get(CONFIG.LOSS),
        metrics=[CONFIG.METRICS])
    return model_custom


class CustomModel_multiview_3(tf.keras.Model):
    """
    This class allows us to create custom model by modifying
    the functions of interest including the train_step test_step
    in order to enable the model to take in 3 layer inputs for
    multiview learning. Also, the execution is switched from
    eager to graph in order to increase the speed of training

    Notes
    -----
    The code is obtained/modified from:

    https://towardsdatascience.com/eager-execution-vs-graph-execution-which-is-better-38162ea4dbf6#:~:text=Eager%20execution%20is%20a%20powerful,they%20occur%20in%20your%20code.

    https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit
    """
    @tf.function
    def train_step(self, data):
        """
        This function modifies the standard train_step in tensorflow
        in order to manipulate and split the input data to put into
        the multiview deep learning model, and graph execution is used instead.
        The function takes in the data and return the corresponding metrics

        Parameters
        ----------
        data : tuple of tf.float32/tf.int

        Returns
        ----------
        The function returns the corresponding metrics
        """
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data
        x1, x2, x3 = tf.split(x,
                              [len(CONFIG.BANDS1),
                               len(CONFIG.BANDS2),
                               len(CONFIG.BANDS3)],
                              3)

        with tf.GradientTape() as tape:
            y_pred = self([x1, x2, x3], training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y,
                                      y_pred,
                                      regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    @tf.function
    def test_step(self, data):
        """
        This function modifies the standard test_step in
        tensorflow in order to manipulate and split the
        input data to put into the multiview deep learning
        model, and graph execution is used instead.
        The function takes in the data and return the
        corresponding metrics

        Parameters
        ----------
        data : tuple of tf.float32/tf.int

        Returns
        ----------
        The function returns the corresponding metrics
        """
        # Unpack the data
        x, y = data
        x1, x2, x3 = tf.split(x,
                              [len(CONFIG.BANDS1),
                               len(CONFIG.BANDS2),
                               len(CONFIG.BANDS3)],
                              3)
        # Compute predictions
        y_pred = self([x1, x2, x3], training=False)
        # Updates the metrics tracking the loss
        self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        # Update the metrics.
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}


def get_model_multiview_3():
    """
    This function puts all the previous mini encoders,
    decoder and conv_block and the modified custom model
    together in order to compile and return a customized
    model for multiview learning with 3 inputs

    Notes
    -----
    The code is obtained/modified from:

    https://towardsdatascience.com/eager-execution-vs-graph-execution-which-is-better-38162ea4dbf6#:~:text=Eager%20execution%20is%20a%20powerful,they%20occur%20in%20your%20code.

    https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit
    """
    # First input
    first_input = layers.Input(shape=[None, None, len(CONFIG.BANDS1)])  # 256
    # Encoder section
    first_encoder0_pool, first_encoder0 = \
        EncoderMiniBlock(first_input, 32)  # 128
    first_encoder1_pool, first_encoder1 = \
        EncoderMiniBlock(first_encoder0_pool, 64)  # 64
    first_encoder2_pool, first_encoder2 = \
        EncoderMiniBlock(first_encoder1_pool, 128)  # 32
    first_encoder3_pool, first_encoder3 = \
        EncoderMiniBlock(first_encoder2_pool, 256)  # 16
    first_encoder4_pool, first_encoder4 = \
        EncoderMiniBlock(first_encoder3_pool, 512)  # 8
    # Center Block
    first_center = conv_block(first_encoder4_pool, 1024)
    # Decoder section
    first_decoder4 = \
        DecoderMiniBlock(first_center, first_encoder4, 512)  # 16
    first_decoder3 = \
        DecoderMiniBlock(first_decoder4, first_encoder3, 256)  # 32
    first_decoder2 = \
        DecoderMiniBlock(first_decoder3, first_encoder2, 128)  # 64
    first_decoder1 = \
        DecoderMiniBlock(first_decoder2, first_encoder1, 64)  # 128
    first_decoder0 = \
        DecoderMiniBlock(first_decoder1, first_encoder0, 32)  # 256

    # Second Input
    second_input = layers.Input(shape=[None, None, len(CONFIG.BANDS2)])  # 256
    # Encoder Section
    second_encoder0_pool, second_encoder0 = \
        EncoderMiniBlock(second_input, 32)  # 128
    second_encoder1_pool, second_encoder1 = \
        EncoderMiniBlock(second_encoder0_pool, 64)  # 64
    second_encoder2_pool, second_encoder2 = \
        EncoderMiniBlock(second_encoder1_pool, 128)  # 32
    second_encoder3_pool, second_encoder3 = \
        EncoderMiniBlock(second_encoder2_pool, 256)  # 16
    second_encoder4_pool, second_encoder4 = \
        EncoderMiniBlock(second_encoder3_pool, 512)  # 8
    # Center block
    second_center = conv_block(second_encoder4_pool, 1024)
    # Decoder section
    second_decoder4 = \
        DecoderMiniBlock(second_center, second_encoder4, 512)  # 16
    second_decoder3 = \
        DecoderMiniBlock(second_decoder4, second_encoder3, 256)  # 32
    second_decoder2 = \
        DecoderMiniBlock(second_decoder3, second_encoder2, 128)  # 64
    second_decoder1 = \
        DecoderMiniBlock(second_decoder2, second_encoder1, 64)  # 128
    second_decoder0 = \
        DecoderMiniBlock(second_decoder1, second_encoder0, 32)  # 256

    # Third input
    third_input = layers.Input(shape=[None, None, len(CONFIG.BANDS3)])  # 256
    # Encoder section
    third_encoder0_pool, third_encoder0 = \
        EncoderMiniBlock(third_input, 32)  # 128
    third_encoder1_pool, third_encoder1 = \
        EncoderMiniBlock(third_encoder0_pool, 64)  # 64
    third_encoder2_pool, third_encoder2 = \
        EncoderMiniBlock(third_encoder1_pool, 128)  # 32
    third_encoder3_pool, third_encoder3 = \
        EncoderMiniBlock(third_encoder2_pool, 256)  # 16
    third_encoder4_pool, third_encoder4 = \
        EncoderMiniBlock(third_encoder3_pool, 512)  # 8
    # Center Block
    third_center = conv_block(third_encoder4_pool, 1024)
    # Decoder Section
    third_decoder4 = \
        DecoderMiniBlock(third_center, third_encoder4, 512)  # 16
    third_decoder3 = \
        DecoderMiniBlock(third_decoder4, third_encoder3, 256)  # 32
    third_decoder2 = \
        DecoderMiniBlock(third_decoder3, third_encoder2, 128)  # 64
    third_decoder1 = \
        DecoderMiniBlock(third_decoder2, third_encoder1, 64)  # 128
    third_decoder0 = \
        DecoderMiniBlock(third_decoder1, third_encoder0, 32)  # 256

    # Fuse two features
    concat_output = tf.keras.layers.concatenate([first_decoder0,
                                                 second_decoder0,
                                                 third_decoder0],
                                                name='cca_output')
    outputs = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(concat_output)

    model_custom = CustomModel_multiview_3([first_input,
                                            second_input,
                                            third_input],
                                           outputs)

    model_custom.compile(
        optimizer=optimizers.get(CONFIG.OPTIMIZER),
        loss=losses.get(CONFIG.LOSS),
        metrics=[CONFIG.METRICS])
    return model_custom


def get_model_multiview_2_HT():
    """
    This function puts all the previous mini encoders,
    decoder and conv_block and the modified custom model
    together in order to compile and return a customized
    model for multiview learning with 2 inputs. This function
    is also used in hyperparameter tuning for loss functions
    and dropouts rate.

    Notes
    -----
    The code is obtained/modified from:

    https://towardsdatascience.com/eager-execution-vs-graph-execution-which-is-better-38162ea4dbf6#:~:text=Eager%20execution%20is%20a%20powerful,they%20occur%20in%20your%20code.

    https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit
    """
    # First input
    first_input = layers.Input(shape=[None, None, len(CONFIG.BANDS1)])  # 256
    # First encoder
    first_encoder0_pool, first_encoder0 = \
        EncoderMiniBlock(first_input,
                         32,
                         dropout_prob=CONFIG.dropout_prob)  # 128
    first_encoder1_pool, first_encoder1 = \
        EncoderMiniBlock(first_encoder0_pool,
                         64,
                         dropout_prob=CONFIG.dropout_prob)  # 64
    first_encoder2_pool, first_encoder2 = \
        EncoderMiniBlock(first_encoder1_pool,
                         128,
                         dropout_prob=CONFIG.dropout_prob)  # 32
    first_encoder3_pool, first_encoder3 = \
        EncoderMiniBlock(first_encoder2_pool,
                         256,
                         dropout_prob=CONFIG.dropout_prob)  # 16
    first_encoder4_pool, first_encoder4 = \
        EncoderMiniBlock(first_encoder3_pool,
                         512,
                         dropout_prob=CONFIG.dropout_prob)  # 8
    # Center Block
    first_center = conv_block(first_encoder4_pool, 1024)  # center
    # First Decoder
    first_decoder4 = \
        DecoderMiniBlock(first_center,
                         first_encoder4,
                         512)  # 16
    first_decoder3 = \
        DecoderMiniBlock(first_decoder4,
                         first_encoder3,
                         256)  # 32
    first_decoder2 = \
        DecoderMiniBlock(first_decoder3,
                         first_encoder2,
                         128)  # 64
    first_decoder1 = \
        DecoderMiniBlock(first_decoder2, first_encoder1, 64)  # 128
    first_decoder0 = \
        DecoderMiniBlock(first_decoder1, first_encoder0, 32)  # 256

    # Second input
    second_input = layers.Input(shape=[None, None, len(CONFIG.BANDS2)])  # 256
    # Second Encoder
    second_encoder0_pool, second_encoder0 = \
        EncoderMiniBlock(second_input, 32)  # 128
    second_encoder1_pool, second_encoder1 = \
        EncoderMiniBlock(second_encoder0_pool, 64)  # 64
    second_encoder2_pool, second_encoder2 = \
        EncoderMiniBlock(second_encoder1_pool, 128)  # 32
    second_encoder3_pool, second_encoder3 = \
        EncoderMiniBlock(second_encoder2_pool, 256)  # 16
    second_encoder4_pool, second_encoder4 = \
        EncoderMiniBlock(second_encoder3_pool, 512)  # 8
    # Center Block
    second_center = conv_block(second_encoder4_pool, 1024)
    # Second Decoder Block
    second_decoder4 = \
        DecoderMiniBlock(second_center, second_encoder4, 512)  # 16
    second_decoder3 = \
        DecoderMiniBlock(second_decoder4, second_encoder3, 256)  # 32
    second_decoder2 = \
        DecoderMiniBlock(second_decoder3, second_encoder2, 128)  # 64
    second_decoder1 = \
        DecoderMiniBlock(second_decoder2, second_encoder1, 64)  # 128
    second_decoder0 = \
        DecoderMiniBlock(second_decoder1, second_encoder0, 32)  # 256

    # Fuse two features
    concat_output = tf.keras.layers.concatenate([first_decoder0,
                                                 second_decoder0],
                                                name='cca_output')
    outputs = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(concat_output)

    model_custom = CustomModel_multiview_2([first_input,
                                            second_input],
                                           outputs)
    model_custom.compile(
        optimizer=optimizers.get(CONFIG.OPTIMIZER),
        loss=CONFIG.LOSS,
        metrics=[CONFIG.METRICS])
    return model_custom


Writing tools/model.py


In [ ]:
%%writefile {PACKAGE_PATH}/config.py

import tensorflow as tf
from . import metrics_

__all__ = ["configuration"]


class configuration:
    """
    In each experiment, the combinations of satellite's bands that is
    used to train the neural network is different. Also the way to train
    the neural network is also different, whether it is feature stack,
    multiview learning with two or three perceptrons. As each experiment
    has different settings, it is important to store them and reuse this
    throughout the project. This class enables user to store the settings
    and reuse the settings.
    """
    def __init__(self, PROJECT_TITLE, BANDS1, TRAIN_SIZE, EVAL_SIZE,
                 BANDS2=[], BANDS3=[], country="TH", image=None, sam_arr=None,
                 type_=1, LOSS="categorical_crossentropy", EPOCHS=10,
                 BATCH_SIZE=16, dropout_prob=0.3):
        """

        Initialising/storing the parameters to use later

        Parameters
        ----------
        PROJECT_TITLE : string
        BANDS1 : list
        TRAIN_SIZE : int/float
        EVAL_SIZE : int/float
        BANDS2 : list
        BANDS3 : list
        country : string
        image : ee.image.Image
        sam_arr : ee.image.Image
        type : int/float

        """
        if type_ == 1:
            self.type_ = "fs"
        elif type_ == 2:
            self.type_ = "m2"
        elif type_ == 3:
            self.type_ = "m3"
        else:
            self.type_ = None
        self.country = country
        self.PROJECT_TITLE = PROJECT_TITLE
        self.BANDS1 = BANDS1
        self.BANDS2 = BANDS2
        self.BANDS3 = BANDS3
        self.BUCKET = "geebucketwater"
        self.FOLDER = f'{self.type_}_{self.country}_Cnn_{self.PROJECT_TITLE}'
        self.TRAIN_SIZE = TRAIN_SIZE
        self.EVAL_SIZE = EVAL_SIZE
        self.BUCKET = "geebucketwater"
        self.TRAINING_BASE = 'training_patches'
        self.EVAL_BASE = 'eval_patches'
        self.TEST_BASE = 'test_patches'
        self.RESPONSE = 'water'
        self.BANDS = BANDS1 + BANDS2 + BANDS3
        self.FEATURES = BANDS1 + BANDS2 + BANDS3 + [self.RESPONSE]
        # Specify the size and shape of patches expected by the model.
        self.KERNEL_SIZE = 256
        self.KERNEL_SHAPE = [self.KERNEL_SIZE, self.KERNEL_SIZE]
        self.COLUMNS = [
            tf.io.FixedLenFeature(shape=self.KERNEL_SHAPE, dtype=tf.float32)
            for k in self.FEATURES
        ]
        self.FEATURES_DICT = dict(zip(self.FEATURES, self.COLUMNS))
        # Specify model training parameters.
        self.BATCH_SIZE = BATCH_SIZE
        self.EPOCHS = EPOCHS
        self.BUFFER_SIZE = 2000
        self.OPTIMIZER = 'adam'
        self.LOSS = LOSS
        self.dropout_prob = dropout_prob
        self.METRICS = ['AUC', "categorical_accuracy", metrics_.f1]
        self.image = image
        self.sam_arr = sam_arr


Writing tools/config.py


In [ ]:
%%writefile {PACKAGE_PATH}/preprocessing.py

import tensorflow as tf
import ee

__all__ = ["Preprocessor", "maskL8sr", "EnsureTwodigit",
           "GenSeasonalDatesMonthly", "getQABits", "cloud_shadows",
           "clouds", "maskClouds", "applyScaleFactors", "changeNames"]


class Preprocessor:
    """
    Class that preprocessese and returns the training,
    evaluation and testing data from google cloud bucket
    """
    def __init__(self, config):
        self.config = config

    def parse_tfrecord(self, example_proto):
        """
        The parsing function Read a serialized example
        into the structure defined by FEATURES_DICT.

        Parameters
        ----------
        example_proto: a serialized Example

        Returns
        ----------
        A dictionary of tensors, keyed by feature name.

        Notes
        -----
        The code is obtained/modified from:

        https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
        """
        return tf.io.parse_single_example(example_proto,
                                          self.config.FEATURES_DICT)

    def to_tuple(self, inputs):
        """
        Function to convert a dictionary of tensors to a
        tuple of (inputs, outputs). Turn the tensors returned
        by parse_tfrecord into a stack in HWC shape.
        Parameters
        ----------
        inputs: A dictionary of tensors, keyed by feature name.

        Returns
        ----------
        A tuple of (inputs, outputs).

        Notes
        -----
        The code is obtained/modified from:

        https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
        """
        inputsList = [inputs.get(key) for key in self.config.FEATURES]
        stacked = tf.stack(inputsList, axis=0)
        # Convert from CHW to HWC
        stacked = tf.transpose(stacked, [1, 2, 0])
        return stacked[:, :, :len(self.config.BANDS)], \
            tf.reshape(tf.one_hot(
                tf.cast(stacked[:, :, len(self.config.BANDS):],
                        tf.int32),
                depth=2), [256, 256, 2])

    def get_dataset(self, pattern):
        """
        Function to read, parse and format to tuple a
        set of input tfrecord files. Get all the files
        matching the pattern, parse and convert to tuple.

        Parameters
        ----------
        pattern: A file pattern to match in a Cloud Storage bucket.

        Returns
        ----------
        A tf.data.Dataset

        Notes
        -----
        The code is obtained/modified from:

        https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
        """
        try:
            glob = tf.io.gfile.glob(pattern)
        except: # noqa
            return "the bucket you specified doesn't exist"
        if glob == []:
            return "the path you specified doesn't have the data"
        dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
        dataset = dataset.map(self.parse_tfrecord, num_parallel_calls=5)
        dataset = dataset.map(self.to_tuple, num_parallel_calls=5)
        return dataset

    def get_training_dataset(self, location):
        """
        Get the preprocessed training dataset
        Parameters
        ----------
        location: string

        Returns
        ----------
        A tf.data.Dataset of training data.

        Notes
        -----
        The code is obtained/modified from:

        https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
        """
        glob = 'gs://' + self.config.BUCKET + \
            '/' + location + "training_patches_" + '*'
        dataset = self.get_dataset(glob)
        dataset = dataset.shuffle(self.config.BUFFER_SIZE).\
            batch(self.config.BATCH_SIZE).\
            repeat()
        return dataset

    def get_training_dataset_for_testing(self, location):
        """
        Get the preprocessed training dataset for testing
        Parameters
        ----------
        location: string

        Returns
        ----------
        A tf.data.Dataset of training data.

        Notes
        -----
        The code is obtained/modified from:

        https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
        """
        glob = 'gs://' + self.config.BUCKET + \
               '/' + location + "training_patches_" + '*'
        dataset = self.get_dataset(glob)
        if type(dataset) == str:
            return dataset
        dataset = dataset.batch(1).repeat()
        return dataset

    def get_eval_dataset(self, location):
        """
        Get the preprocessed evaluation dataset
        Parameters
        ----------
        location: string

        Returns
        ----------
        A tf.data.Dataset of evaluation data.

        Notes
        -----
        The code is obtained/modified from:

        https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
        """
        glob = 'gs://' + self.config.BUCKET + \
               '/' + location + "eval_patches_" + '*'
        dataset = self.get_dataset(glob)
        if type(dataset) == str:
            return dataset
        dataset = dataset.batch(1).repeat()
        return dataset

    # print(iter(evaluation.take(1)).next())

    def get_test_dataset(self, location, test_base):
        """
        Get the preprocessed testing dataset
        Parameters
        ----------
        location: string

        Returns
        ----------
        A tf.data.Dataset of testing data.

        Notes
        -----
        The code is obtained/modified from:

        https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
        """
        glob = 'gs://' + self.config.BUCKET + \
               '/' + location + test_base + '*'
        dataset = self.get_dataset(glob)
        if type(dataset) == str:
            return dataset
        dataset = dataset.batch(1).repeat()
        return dataset


def maskL8sr(image):
    """
    Get the landsat-8 image and returned a cloud masked image
    ----------
    image: ee.image.Image

    Returns
    ----------
    A maksed landsat-8 ee.image.Image

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
    cloudShadowBitMask = ee.Number(2).pow(3).int()
    cloudsBitMask = ee.Number(2).pow(5).int()
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
        qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask).select(BANDS).divide(10000)


def EnsureTwodigit(number):
    """
    Transform the input month into string in the
    correct format for date and time.
    ----------
    number: int

    Returns
    ----------
    months in string.

    """
    if number > 12:
        return str(12)
    if number < 10:
        return "0" + str(number)
    else:
        return str(number)


def GenSeasonalDatesMonthly(start, end, month_frequency=3):
    """
    Given two dictionary containing the key month and year,
    return two arrays that contains the time between the
    interval of start and end.
    ----------
    start: dict
    end: dict

    Returns
    ----------
    Two arrays containing the time elapsed between start and end

    """
    diff_year = end["year"] - start["year"]
    diff_month = end["month"] - start["month"]
    starts = []
    ends = []
    first_data = str(start["year"]) + "-" + \
        EnsureTwodigit(start["month"]) + "-01"
    if diff_year > 0:
        return "please insert the same year"
    else:
        for i in range(round(diff_month / month_frequency)):
            first_data = str(start["year"]) + "-" + \
                EnsureTwodigit(start["month"] + month_frequency * i) + "-01"
            second_data = str(start["year"]) + "-" + \
                EnsureTwodigit(start["month"] +
                               month_frequency *
                               i +
                               month_frequency) + "-01"
            starts.append(first_data)
            ends.append(second_data)
    return starts, ends


# As collection 1 of Landsat-8 ceased at
# December 2021, collection 2 must be used instead


def getQABits(image, start, end, newName):
    """
    Compute the bits we need to extract.
    ----------
    image: ee.image.Image
    start: int
    end: int
    newName: string

    Returns
    ----------
    Return a single band image of the extracted QA bits
    with a new name

    Notes
    ----------
    Code is modified from
    https://gis.stackexchange.com/questions/277059/cloud-mask-for-landsat8-on-google-earth-engine
    """
    pattern = 0
    for i in range(start, end + 1):
        pattern += 2**i
    return image.select([0], [newName])\
                .bitwiseAnd(pattern)\
                .rightShift(start)


def cloud_shadows(image):
    """
    return the masked cloud shadow image from QABits image.
    ----------
    image: ee.image.Image

    Returns
    ----------
    Return an image masking out cloudy areas.

    Notes
    -----
    Code is modified from
    https://gis.stackexchange.com/questions/277059/cloud-mask-for-landsat8-on-google-earth-engine
    """
    # Select the QA band.
    QA = image.select(['QA_PIXEL'])
    # Get the internal_cloud_algorithm_flag bit.
    return getQABits(QA, 3, 3, 'cloud_shadows').eq(0)


def clouds(image):
    """
    Mask out cloudy pixels from QABit image.
    ----------
    image: ee.image.Image

    Returns
    ----------
    Return an image masking out cloudy areas.

    Notes
    -----
    Code is modified from
    https://gis.stackexchange.com/questions/277059/cloud-mask-for-landsat8-on-google-earth-engine
    """
    # Select the QA band.
    QA = image.select(['QA_PIXEL'])
    # Get the internal_cloud_algorithm_flag bit.
    return getQABits(QA, 5, 5, 'Cloud').eq(0)
    # Return an image masking out cloudy areas.


def maskClouds(image):
    """
    Put all the functions together to mask the clouds and
    shadows
    ----------
    image: ee.image.Image

    Returns
    ----------
    Return an image masking out cloudy and shadow area.

    Notes
    -----
    Code is modified from
    https://gis.stackexchange.com/questions/277059/cloud-mask-for-landsat8-on-google-earth-engine
    """
    cs = cloud_shadows(image)
    c = clouds(image)
    image = image.updateMask(cs)
    return image.updateMask(c)


def applyScaleFactors(image):
    """
    Adjust scale factor to standardize the visualization
    ----------
    image: ee.image.Image

    Returns
    ----------
    Adjusted image with correct scale factor

    Notes
    -----
    Code is modified from
    https://gis.stackexchange.com/questions/277059/cloud-mask-for-landsat8-on-google-earth-engine
    """
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True)\
                .addBands(thermalBands, None, True)


def changeNames(image):
    """
    Adjust bandNames of collection 2 to match collection 1
    ----------
    image: ee.image.Image

    Returns
    ----------
    ee.image.Image with adjusted bandNames
    """
    return image.select(['SR_B1', 'SR_B2', 'SR_B3',
                         'SR_B4', 'SR_B5', 'SR_B6',
                         'SR_B7', 'SR_QA_AEROSOL',
                         'ST_B10', 'ST_ATRAN', 'ST_CDIST',
                         'ST_DRAD', 'ST_EMIS', 'ST_EMSD',
                         'ST_QA', 'ST_TRAD', 'ST_URAD',
                         'QA_PIXEL', 'QA_RADSAT'],
                        ['B1', 'B2', 'B3', 'B4', 'B5',
                         'B6', 'B7', 'SR_QA_AEROSOL',
                         'ST_B10', 'ST_ATRAN', 'ST_CDIST',
                         'ST_DRAD', 'ST_EMIS', 'ST_EMSD',
                         'ST_QA', 'ST_TRAD', 'ST_URAD',
                         'QA_PIXEL', 'QA_RADSAT'])


Writing tools/preprocessing.py


## Authentication

Authentication with google colab, earth engine api and google cloud bucket is required before proceeding.

In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

project_id = 'coastal-cell-299117'
!gcloud config set project {project_id}

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Jje208FHNudzagVPi3Z5UmId0jEyumH_LhivpoPQaxg&tc=29PJa9hjPBo4h-VNWp-crFh4Kyx5jeSPahN_NXFqcaA&cc=GtfHIhc-g-2JGXYjSF25CwFFYDG01WMzinz_LEgCYBE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qhCObOYGr9kAZKQCBdFBlzDLo-T7-_WNqEYraa2tRO45JGLddCtNrY

Successfully saved authorization token.
Updated property [core/project].


In [ ]:
%%capture
!pip install wandb --upgrade

import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
import time
import pandas as pd
from tools import preprocessing, config, metrics_, model
# from importlib import reload
# reload(preprocessing) # Uncomment this line to rerun the modified packages

## Setting up Config - to train globally

In total there are:
- 32 feature stack deep learning experiments
- 37 Multiview with 2 input experiments
- 8 Multiview with 3 input experiments

Each experiment has a different configuration for example, different experiment name, bands for each input layer. Hence, a configuration is neccesary.

In [ ]:
configs_fs_global = {}
train_size = 72 * 10
eval_size = 72 * 3

# Feature stack experiment

configs_fs_global["S1A_el_sl_as"] = \
    config.configuration("S1A_el_sl_as",
                         ["VV", "VH", "angle", "elevation",
                          "slope", "aspect"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["S1A_el"] = \
    config.configuration("S1A_el",
                         ["VV", "VH", "angle", "elevation"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["S1A_sl"] = \
    config.configuration("S1A_sl",
                         ["VV", "VH", "angle", "slope"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["S1A_as"] = \
    config.configuration("S1A_as",
                         ["VV", "VH", "angle", "aspect"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["S1A_sl_as"] = \
    config.configuration("S1A_sl_as",
                         ["VV", "VH", "angle", "slope", "aspect"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["S1A_el_sl"] = \
    config.configuration("S1A_el_sl",
                         ["VV", "VH", "angle", "elevation", "slope"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["S1A_el_as"] = \
    config.configuration("S1A_el_as",
                         ["VV", "VH", "angle", "elevation", "aspect"],
                         train_size,
                         eval_size, country="global")

configs_fs_global["S1_el_sl_as"] = \
    config.configuration("S1_el_sl_as",
                         ["VV", "VH", "elevation", "slope", "aspect"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["S1_el"] = \
    config.configuration("S1_el",
                         ["VV", "VH", "elevation"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["S1_sl"] = \
    config.configuration("S1_sl",
                         ["VV", "VH", "slope"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["S1_as"] = \
    config.configuration("S1_as",
                         ["VV", "VH", "aspect"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["S1_sl_as"] = \
    config.configuration("S1_sl_as",
                         ["VV", "VH", "slope", "aspect"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["S1_el_sl"] = \
    config.configuration("S1_el_sl",
                         ["VV", "VH", "elevation", "slope"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["S1_el_as"] = \
    config.configuration("S1_el_as",
                         ["VV", "VH", "elevation", "aspect"],
                         train_size,
                         eval_size, country="global")

configs_fs_global["L8SR_el_sl_as"] = \
    config.configuration("L8SR_el_sl_as",
                         ["B2", "B3", "B4", "B5", "B6", "B7",
                          "elevation", "slope", "aspect"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["L8SR_el"] = \
    config.configuration("L8SR_el",
                         ["B2", "B3", "B4", "B5", "B6", "B7",
                          "elevation"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["L8SR_sl"] = \
    config.configuration("L8SR_sl",
                         ["B2", "B3", "B4", "B5", "B6",
                          "B7", "slope"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["L8SR_as"] = \
    config.configuration("L8SR_as",
                         ["B2", "B3", "B4", "B5", "B6",
                          "B7", "aspect"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["L8SR_sl_as"] = \
    config.configuration("L8SR_sl_as",
                         ["B2", "B3", "B4", "B5", "B6", "B7",
                          "slope", "aspect"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["L8SR_el_sl"] = \
    config.configuration("L8SR_el_sl",
                         ["B2", "B3", "B4", "B5", "B6", "B7",
                          "elevation", "slope"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["L8SR_el_as"] = \
    config.configuration("L8SR_el_as",
                         ["B2", "B3", "B4", "B5", "B6", "B7",
                          "elevation", "aspect"],
                         train_size,
                         eval_size, country="global")

configs_fs_global["L8SR_S1_el"] = \
    config.configuration("L8SR_S1_el",
                         ["B2", "B3", "B4", "B5", "B6", "B7",
                          "VV", "VH", "elevation"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["L8SR_S1_sl"] = \
    config.configuration("L8SR_S1_sl",
                         ["B2", "B3", "B4", "B5", "B6", "B7",
                          "VV", "VH", "slope"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["L8SR_S1_sl_el_as"] = \
    config.configuration("L8SR_S1_sl_el_as",
                         ["B2", "B3", "B4", "B5", "B6", "B7",
                          "VV", "VH", "slope", "elevation", "aspect"],
                         train_size,
                         eval_size, country="global")

configs_fs_global["L8SR_S1A_el"] = \
    config.configuration("L8SR_S1A_el",
                         ["B2", "B3", "B4", "B5", "B6", "B7",
                          "VV", "VH", "angle", "elevation"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["L8SR_S1A_sl"] = \
    config.configuration("L8SR_S1A_sl",
                         ["B2", "B3", "B4", "B5", "B6", "B7",
                          "VV", "VH", "angle", "slope"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["L8SR_S1A_sl_el_as"] = \
    config.configuration("L8SR_S1A_sl_el_as",
                         ["B2", "B3", "B4", "B5", "B6", "B7",
                          "VV", "VH", "angle", "slope", "elevation", "aspect"],
                         train_size,
                         eval_size, country="global")

configs_fs_global["L8SR"] = \
    config.configuration("L8SR",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["S1"] = \
    config.configuration("S1",
                         ["VV", "VH"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["S1A"] = \
    config.configuration("S1A",
                         ["VV", "VH", "angle"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["L8SR_S1"] = \
    config.configuration("L8SR_S1",
                         ["B2", "B3", "B4", "B5", "B6",
                          "B7", "VV", "VH"],
                         train_size,
                         eval_size, country="global")
configs_fs_global["L8SR_S1A"] = \
    config.configuration("L8SR_S1A",
                         ["B2", "B3", "B4", "B5", "B6",
                          "B7", "VV", "VH", "angle"],
                         train_size,
                         eval_size, country="global")

# Multiexperiment

configs_multi_global = {}

configs_multi_global["S1A_el_sl_as"] = \
    config.configuration("S1A_el_sl_as",
                         ["VV", "VH", "angle"],
                         train_size,
                         eval_size,
                         ["elevation", "slope", "aspect"],
                         type_=2,
                         country="global")
configs_multi_global["S1A_el"] = \
    config.configuration("S1A_el",
                         ["VV", "VH", "angle"],
                         train_size,
                         eval_size,
                         ["elevation"],
                         type_=2,
                         country="global")
configs_multi_global["S1A_sl"] = \
    config.configuration("S1A_sl",
                         ["VV", "VH", "angle"],
                         train_size,
                         eval_size,
                         ["slope"],
                         type_=2,
                         country="global")
configs_multi_global["S1A_as"] = \
    config.configuration("S1A_as",
                         ["VV", "VH", "angle"],
                         train_size,
                         eval_size,
                         ["aspect"],
                         type_=2,
                         country="global")
configs_multi_global["S1A_sl_as"] = \
    config.configuration("S1A_sl_as",
                         ["VV", "VH", "angle"],
                         train_size,
                         eval_size,
                         ["slope", "aspect"],
                         type_=2,
                         country="global")
configs_multi_global["S1A_el_sl"] = \
    config.configuration("S1A_el_sl",
                         ["VV", "VH", "angle"],
                         train_size,
                         eval_size,
                         ["elevation", "slope"],
                         type_=2,
                         country="global")
configs_multi_global["S1A_el_as"] = \
    config.configuration("S1A_el_as",
                         ["VV", "VH", "angle"],
                         train_size,
                         eval_size,
                         ["elevation", "aspect"],
                         type_=2,
                         country="global")

configs_multi_global["S1_el_sl_as"] = \
    config.configuration("S1_el_sl_as",
                         ["VV", "VH"],
                         train_size,
                         eval_size,
                         ["elevation", "slope", "aspect"],
                         type_=2,
                         country="global")
configs_multi_global["S1_el"] = \
    config.configuration("S1_el",
                         ["VV", "VH"],
                         train_size,
                         eval_size,
                         ["elevation"],
                         type_=2,
                         country="global")
configs_multi_global["S1_sl"] = \
    config.configuration("S1_sl",
                         ["VV", "VH"],
                         train_size,
                         eval_size,
                         ["slope"],
                         type_=2,
                         country="global")
configs_multi_global["S1_as"] = \
    config.configuration("S1_as",
                         ["VV", "VH"],
                         train_size,
                         eval_size,
                         ["aspect"],
                         type_=2,
                         country="global")
configs_multi_global["S1_sl_as"] = \
    config.configuration("S1_sl_as",
                         ["VV", "VH"],
                         train_size,
                         eval_size,
                         ["slope", "aspect"],
                         type_=2,
                         country="global")
configs_multi_global["S1_el_sl"] = \
    config.configuration("S1_el_sl",
                         ["VV", "VH"],
                         train_size,
                         eval_size,
                         ["elevation", "slope"],
                         type_=2,
                         country="global")
configs_multi_global["S1_el_as"] = \
    config.configuration("S1_el_as",
                         ["VV", "VH"],
                         train_size,
                         eval_size,
                         ["elevation", "aspect"],
                         type_=2,
                         country="global")

configs_multi_global["L8SR_el_sl_as"] = \
    config.configuration("L8SR_el_sl_as",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["elevation", "slope", "aspect"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_el"] = \
    config.configuration("L8SR_el",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["elevation"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_sl"] = \
    config.configuration("L8SR_sl",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["slope"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_as"] = \
    config.configuration("L8SR_as",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["aspect"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_sl_as"] = \
    config.configuration("L8SR_sl_as",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["slope", "aspect"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_el_sl"] = \
    config.configuration("L8SR_el_sl",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["elevation", "slope"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_el_as"] = \
    config.configuration("L8SR_el_as",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["elevation", "aspect"],
                         type_=2,
                         country="global")


configs_multi_global["L8SR_S1_as"] = \
    config.configuration("L8SR_S1_as",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "aspect"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_S1_el"] = \
    config.configuration("L8SR_S1_el",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "elevation"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_S1_sl"] = \
    config.configuration("L8SR_S1_sl",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "slope"],
                         type_=2,
                         country="global")

configs_multi_global["L8SR_S1_sl_as"] = \
    config.configuration("L8SR_S1_sl_as",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "slope", "aspect"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_S1_el_sl"] = \
    config.configuration("L8SR_S1_el_sl",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "elevation", "slope"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_S1_el_as"] = \
    config.configuration("L8SR_S1_el_as",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "elevation", "aspect"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_S1_sl_el_as"] = \
    config.configuration("L8SR_S1_sl_el_as",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "slope", "elevation", "aspect"],
                         type_=2,
                         country="global")

configs_multi_global["L8SR_S1A_as"] = \
    config.configuration("L8SR_S1A_as",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "angle", "aspect"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_S1A_el"] = \
    config.configuration("L8SR_S1A_el",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "angle", "elevation"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_S1A_sl"] = \
    config.configuration("L8SR_S1A_sl",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "angle", "slope"],
                         type_=2,
                         country="global")

configs_multi_global["L8SR_S1A_sl_as"] = \
    config.configuration("L8SR_S1A_sl_as",
                         ["B2", "B3", "B4", "B5",
                          "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "angle",
                          "slope", "aspect"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_S1A_el_sl"] = \
    config.configuration("L8SR_S1A_el_sl",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "angle",
                          "elevation", "slope"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_S1A_el_as"] = \
    config.configuration("L8SR_S1A_el_as",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "angle",
                          "elevation", "aspect"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_S1A_sl_el_as"] = \
    config.configuration("L8SR_S1A_sl_el_as",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "angle", "slope",
                          "elevation", "aspect"],
                         type_=2,
                         country="global")

configs_multi_global["L8SR_S1"] = \
    config.configuration("L8SR_S1",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH"],
                         type_=2,
                         country="global")
configs_multi_global["L8SR_S1A"] = \
    config.configuration("L8SR_S1A",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "angle"],
                         type_=2,
                         country="global")

configs_multi_3_global = {}

configs_multi_3_global["L8SR_S1_as3"] = \
    config.configuration("L8SR_S1_as3",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH"],
                         ["aspect"],
                         type_=3,
                         country="global")
configs_multi_3_global["L8SR_S1_el3"] = \
    config.configuration("L8SR_S1_el3",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH"],
                         ["elevation"],
                         type_=3,
                         country="global")
configs_multi_3_global["L8SR_S1_sl3"] = \
    config.configuration("L8SR_S1_sl3",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH"],
                         ["slope"],
                         type_=3,
                         country="global")
configs_multi_3_global["L8SR_S1_sl_el_as3"] = \
    config.configuration("L8SR_S1_sl_el_as3",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH"],
                         ["slope", "elevation", "aspect"],
                         type_=3,
                         country="global")

configs_multi_3_global["L8SR_S1A_as3"] = \
    config.configuration("L8SR_S1A_as3",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "angle"],
                         ["aspect"],
                         type_=3,
                         country="global")
configs_multi_3_global["L8SR_S1A_el3"] = \
    config.configuration("L8SR_S1A_el3",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "angle"],
                         ["elevation"],
                         type_=3,
                         country="global")
configs_multi_3_global["L8SR_S1A_sl3"] = \
    config.configuration("L8SR_S1A_sl3",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "angle"],
                         ["slope"],
                         type_=3,
                         country="global")
configs_multi_3_global["L8SR_S1A_sl_el_as3"] = \
    config.configuration("L8SR_S1A_sl_el_as3",
                         ["B2", "B3", "B4", "B5", "B6", "B7"],
                         train_size,
                         eval_size,
                         ["VV", "VH", "angle"],
                         ["slope", "elevation", "aspect"], type_=3)


# Feature stack


<b> The model </b>: Here we use the Keras implementation of the U-Net model. The U-Net model takes 256x256 pixel patches as input with the interested feature of each experiment and outputs per-pixel class probability. We will use categorical cross entropy as loss function and softmax as the last layer because we are working on a classification problem.

For each experiment, we will:
- Load the training and evaluation dataset from google cloud bucket into a `tf.data.Dataset`.
- Train the Feature stack deep learning UNET model for 10 epochs
- Store the trained model in google cloud bucket for future prediction
- Store the losses and metrics using Wandb.ai

<b> Note:</b> The time to train the model could take up to 4-5 hours because we are training 32 UNET model that is not pretrained, but this is within the colab background execution runtime. If the connection cuts off, the experiment is continued where the training stops.

In [ ]:
config = {
    "architecture": "Unet",
    "epochs": 10,
    "batch_size": 16
}

df = pd.DataFrame(columns=['Name', 'Loss', 'val_loss',
                           "auc", "val_auc", "accuracy",
                           "val_accuracy", "f1", "val_f1"])

for i in range(0, len(list(configs_fs_global))):
    run = wandb.init(project='kl-121-dissertation', reinit=True, config=config)
    print(i + 1, " loaded \n")
    conf = configs_fs_global[list(configs_fs_global)[i]]
    print(conf.PROJECT_TITLE, conf.type_)
    preproc = preprocessing.Preprocessor(conf)
    training = preproc.get_training_dataset("train_in_global/")
    evaluation = preproc.get_eval_dataset("train_in_global/")
    model.CONFIG = conf
    EPOCHS = 10
    wandb.run.name = "Feature Stack" + conf.PROJECT_TITLE + \
        conf.country + "training"
    start = time.time()
    EPO = [i for i in range(1, EPOCHS + 1)]
    model_custom = model.get_model()

    history = model_custom.fit(
        x=training,
        epochs=EPOCHS,
        steps_per_epoch=int(72 * 10 / conf.BATCH_SIZE),
        validation_data=evaluation,
        validation_steps=72 * 3,
        callbacks=[WandbCallback()]
    )
    end = time.time()
    print(f'Time for {EPOCHS} epochs is: ', end - start)
    Model_name = conf.PROJECT_TITLE + "_EPOCHS_" + str(EPOCHS)
    MODEL_DIR = 'gs://' + conf.BUCKET + "/" + \
        conf.FOLDER + "/Models/" + Model_name
    model_custom.save(MODEL_DIR, save_format='tf')
    hist_keys = [*history.history]

    df.loc[i] = [conf.PROJECT_TITLE] + \
        [history.history["loss"][-1]] + [history.history["val_loss"][-1]] + \
        [history.history["auc"][-1]] + [history.history["val_auc"][-1]] + \
        [history.history["categorical_accuracy"][-1]] + \
        [history.history["val_categorical_accuracy"][-1]] + \
        [history.history["f1"][-1]] + [history.history["val_f1"][-1]] \

    wandb.log({'global_feature_stack_train_eval': df})
    run.finish()

1  loaded 

S1A_el_sl_as fs
Epoch 1/10
45/45 [==============================] - 303s 4s/step - loss: 28.5250 - auc: 0.9368 - categorical_accuracy: 0.9305 - f1: 0.9313 - val_loss: 0.6497 - val_auc: 0.9792 - val_categorical_accuracy: 0.9759 - val_f1: 0.9759 - _timestamp: 1658358754.0000 - _runtime: 309.0000
Epoch 2/10
45/45 [==============================] - 119s 3s/step - loss: 1.6013 - auc: 0.9591 - categorical_accuracy: 0.9432 - f1: 0.9432 - val_loss: 0.2943 - val_auc: 0.9797 - val_categorical_accuracy: 0.9710 - val_f1: 0.9710 - _timestamp: 1658358873.0000 - _runtime: 428.0000
Epoch 3/10
45/45 [==============================] - 79s 2s/step - loss: 0.3611 - auc: 0.9801 - categorical_accuracy: 0.9643 - f1: 0.9642 - val_loss: 0.2534 - val_auc: 0.9836 - val_categorical_accuracy: 0.9799 - val_f1: 0.9799 - _timestamp: 1658358951.0000 - _runtime: 506.0000
Epoch 4/10
45/45 [==============================] - 123s 3s/step - loss: 0.2794 - auc: 0.9823 - categorical_accuracy: 0.9700 - f1: 0.9706 

auc,▁▄▆▇▇█████
categorical_accuracy,▁▃▅▆▇█████
epoch,▁▂▃▃▄▅▆▆▇█
f1,▁▃▅▆▇█▇█▇█
loss,█▁▁▁▁▁▁▁▁▁
val_auc,▂▃▆▁▁▂▄▇▃█
val_categorical_accuracy,▄▁▇▅▅▇████
val_f1,▄▁▇▅▅▇████
val_loss,█▃▃▃▂▂▁▁▁▁
GFLOPS,0.0
auc,0.99202


2  loaded 

S1A_el fs
Epoch 1/10
45/45 [==============================] - 68s 951ms/step - loss: 56.9772 - auc: 0.9212 - categorical_accuracy: 0.8970 - f1: 0.8969 - val_loss: 0.2699 - val_auc: 0.9860 - val_categorical_accuracy: 0.9807 - val_f1: 0.9807 - _timestamp: 1658359515.0000 - _runtime: 74.0000
Epoch 2/10
45/45 [==============================] - 42s 949ms/step - loss: 2.9954 - auc: 0.9520 - categorical_accuracy: 0.9186 - f1: 0.9187 - val_loss: 0.1805 - val_auc: 0.9854 - val_categorical_accuracy: 0.9819 - val_f1: 0.9819 - _timestamp: 1658359557.0000 - _runtime: 116.0000
Epoch 3/10
45/45 [==============================] - 40s 896ms/step - loss: 1.8370 - auc: 0.9581 - categorical_accuracy: 0.9195 - f1: 0.9203 - val_loss: 0.1003 - val_auc: 0.9886 - val_categorical_accuracy: 0.9817 - val_f1: 0.9817 - _timestamp: 1658359597.0000 - _runtime: 156.0000
Epoch 4/10
45/45 [==============================] - 62s 1s/step - loss: 1.5357 - auc: 0.9509 - categorical_accuracy: 0.9172 - f1: 0.9145 -

auc,▁▄▅▄▆▆▇▇▇█
categorical_accuracy,▁▃▃▃▅▆▇▇██
epoch,▁▂▃▃▄▅▆▆▇█
f1,▁▃▃▂▅▆▇▇██
loss,█▁▁▁▁▁▁▁▁▁
val_auc,▃▂▄▆█▆▄▆▂▁
val_categorical_accuracy,▁▃▂▂▃▃▅▅█▄
val_f1,▁▃▂▂▃▃▅▅█▄
val_loss,█▅▂▁▁▂▁▁▂▂
GFLOPS,0.0
auc,0.99232


3  loaded 

S1A_sl fs
Epoch 1/10
45/45 [==============================] - 65s 893ms/step - loss: 3.1246 - auc: 0.9502 - categorical_accuracy: 0.9292 - f1: 0.9297 - val_loss: 0.0636 - val_auc: 0.9975 - val_categorical_accuracy: 0.9855 - val_f1: 0.9855 - _timestamp: 1658360036.0000 - _runtime: 72.0000
Epoch 2/10
45/45 [==============================] - 39s 879ms/step - loss: 0.1186 - auc: 0.9902 - categorical_accuracy: 0.9643 - f1: 0.9627 - val_loss: 0.0510 - val_auc: 0.9980 - val_categorical_accuracy: 0.9878 - val_f1: 0.9878 - _timestamp: 1658360074.0000 - _runtime: 110.0000
Epoch 3/10
45/45 [==============================] - 38s 865ms/step - loss: 0.0855 - auc: 0.9939 - categorical_accuracy: 0.9748 - f1: 0.9752 - val_loss: 0.0474 - val_auc: 0.9974 - val_categorical_accuracy: 0.9878 - val_f1: 0.9878 - _timestamp: 1658360112.0000 - _runtime: 148.0000
Epoch 4/10
45/45 [==============================] - 61s 1s/step - loss: 0.0757 - auc: 0.9947 - categorical_accuracy: 0.9786 - f1: 0.9789 - 

...

auc,▁▇████████
categorical_accuracy,▁▆█▇███▇▇▇
epoch,▁▂▃▃▄▅▆▆▇█
f1,▁▆█▇███▇▇▇
loss,█▁▁▁▁▁▁▁▁▁
val_auc,▄▆▇▁▇▅████
val_categorical_accuracy,▁▂▅▂█▂▇▅▁▁
val_f1,▁▂▅▂█▂▇▅▁▁
val_loss,▇▃▃█▁▄▁▂▅▅
GFLOPS,0.0
auc,0.99337


31  loaded 

L8SR_S1 fs
Epoch 1/10
45/45 [==============================] - 64s 887ms/step - loss: 0.7007 - auc: 0.9317 - categorical_accuracy: 0.9396 - f1: 0.9404 - val_loss: 0.0939 - val_auc: 0.9854 - val_categorical_accuracy: 0.9821 - val_f1: 0.9821 - _timestamp: 1658374670.0000 - _runtime: 72.0000
Epoch 2/10
45/45 [==============================] - 39s 892ms/step - loss: 0.1520 - auc: 0.9824 - categorical_accuracy: 0.9669 - f1: 0.9675 - val_loss: 0.0811 - val_auc: 0.9888 - val_categorical_accuracy: 0.9846 - val_f1: 0.9846 - _timestamp: 1658374709.0000 - _runtime: 111.0000
Epoch 3/10
45/45 [==============================] - 38s 867ms/step - loss: 0.1327 - auc: 0.9842 - categorical_accuracy: 0.9753 - f1: 0.9757 - val_loss: 0.0899 - val_auc: 0.9841 - val_categorical_accuracy: 0.9815 - val_f1: 0.9815 - _timestamp: 1658374749.0000 - _runtime: 151.0000
Epoch 4/10
45/45 [==============================] - 71s 2s/step - loss: 0.1129 - auc: 0.9869 - categorical_accuracy: 0.9758 - f1: 0.9750 

auc,▁▆▇▇▇▇▇▇██
categorical_accuracy,▁▅▆▆█▇▇▇▇█
epoch,▁▂▃▃▄▅▆▆▇█
f1,▁▅▆▆▇▇▇▇▇█
loss,█▂▂▂▁▂▁▁▁▁
val_auc,▂▄▁▄▆█████
val_categorical_accuracy,▄▆▄▆▇▇▇▇█▁
val_f1,▄▆▄▆▇▇▇▇█▁
val_loss,█▆▇█▄▂▂▂▁▅
GFLOPS,0.0
auc,0.99816


32  loaded 

L8SR_S1A fs
Epoch 1/10
45/45 [==============================] - 64s 880ms/step - loss: 1.4213 - auc: 0.9519 - categorical_accuracy: 0.9326 - f1: 0.9324 - val_loss: 0.0425 - val_auc: 0.9981 - val_categorical_accuracy: 0.9817 - val_f1: 0.9817 - _timestamp: 1658375211.0000 - _runtime: 72.0000
Epoch 2/10
45/45 [==============================] - 38s 856ms/step - loss: 0.1114 - auc: 0.9914 - categorical_accuracy: 0.9630 - f1: 0.9635 - val_loss: 0.0479 - val_auc: 0.9987 - val_categorical_accuracy: 0.9871 - val_f1: 0.9871 - _timestamp: 1658375250.0000 - _runtime: 111.0000
Epoch 3/10
45/45 [==============================] - 39s 876ms/step - loss: 0.0889 - auc: 0.9937 - categorical_accuracy: 0.9722 - f1: 0.9726 - val_loss: 0.0364 - val_auc: 0.9989 - val_categorical_accuracy: 0.9913 - val_f1: 0.9913 - _timestamp: 1658375287.0000 - _runtime: 148.0000
Epoch 4/10
45/45 [==============================] - 60s 1s/step - loss: 0.0702 - auc: 0.9952 - categorical_accuracy: 0.9801 - f1: 0.9807

auc,▁▇▇▇██████
categorical_accuracy,▁▅▆▇▇▇████
epoch,▁▂▃▃▄▅▆▆▇█
f1,▁▅▆▇▇▇████
loss,█▁▁▁▁▁▁▁▁▁
val_auc,▅▇█▃▇▅▁█▁▂
val_categorical_accuracy,▁▅▇▅██████
val_f1,▁▅▇▅██████
val_loss,▅█▂▅▂▃▂▃▂▁
GFLOPS,0.0
auc,0.99889


# Multiview-2


<b> The model </b>: Here we use the Keras implementation of the U-Net model. The modified U-Net model takes in two layers of inputs and each has a structure of UNET and and outputs per-pixel class probability. We will use categorical cross entropy as loss function and softmax as the last layer because we are working on a classification problem.

For each experiment, we will:
- Load the training and evaluation dataset from google cloud bucket into a `tf.data.Dataset`.
- Train the Multiview deep learning with 2 inputs  UNET model for 10 epochs
- Store the trained model in google cloud bucket for future prediction
- Store the losses and metrics using Wandb.ai

<b> Note:</b> The time to train the model could take up to 4-5 hours because we are training 37 UNET model that is not pretrained, but this is within the colab background execution runtime. If the connection cuts off, the experiment is continued where the training stops.

In [ ]:
config = {
    "architecture": "Unet",
    "epochs": 10,
    "batch_size": 16
}

df = pd.DataFrame(columns=['Name', 'Loss', 'val_loss',
                           "auc", "val_auc", "accuracy",
                           "val_accuracy", "f1", "val_f1"])

for i in range(0, len(list(configs_multi_global))):
    run = wandb.init(project='kl-121-dissertation', reinit=True, config=config)
    print(i + 1, " loaded \n")
    conf = configs_multi_global[list(configs_multi_global)[i]]
    print(conf.PROJECT_TITLE, conf.type_)
    preproc = preprocessing.Preprocessor(conf)
    training = preproc.get_training_dataset("train_in_global/")
    evaluation = preproc.get_eval_dataset("train_in_global/")
    model.CONFIG = conf
    EPOCHS = 10
    wandb.run.name = "Multiview_2" + conf.PROJECT_TITLE + conf.country
    start = time.time()
    EPO = [i for i in range(1, EPOCHS + 1)]
    model_custom = model.get_model_multiview_2()

    history = model_custom.fit(
        x=training,
        epochs=EPOCHS,
        steps_per_epoch=int(72 * 10 / conf.BATCH_SIZE),
        validation_data=evaluation,
        validation_steps=72 * 3,
        callbacks=[WandbCallback()]
    )
    end = time.time()
    print(f'Time for {EPOCHS} epochs is: ', end - start)
    Model_name = conf.PROJECT_TITLE + "_EPOCHS_" + str(EPOCHS)
    MODEL_DIR = 'gs://' + conf.BUCKET + "/" \
        + conf.FOLDER + "/Models/" + Model_name
    model_custom.save(MODEL_DIR, save_format='tf')
    hist_keys = [*history.history]

    df.loc[i] = [conf.PROJECT_TITLE] + \
        [history.history["loss"][-1]] + [history.history["val_loss"][-1]] + \
        [history.history["auc"][-1]] + [history.history["val_auc"][-1]] + \
        [history.history["categorical_accuracy"][-1]] + \
        [history.history["val_categorical_accuracy"][-1]] + \
        [history.history["f1"][-1]] + [history.history["val_f1"][-1]] \

    wandb.log({'Multiview_data_2_global_final': df})
    run.finish()

wandb: Currently logged in as: kl-121 (kl-121-dissertation). Use `wandb login --relogin` to force relogin


1  loaded 

S1A_el_sl_as m2
Epoch 1/10
45/45 [==============================] - 92s 1s/step - loss: 47.9772 - auc: 0.9329 - categorical_accuracy: 0.9253 - f1: 0.9261 - val_loss: 0.3647 - val_auc: 0.9844 - val_categorical_accuracy: 0.9736 - val_f1: 0.9736 - _timestamp: 1658399830.0000 - _runtime: 101.0000
Epoch 2/10
45/45 [==============================] - 62s 1s/step - loss: 1.5406 - auc: 0.9684 - categorical_accuracy: 0.9487 - f1: 0.9493 - val_loss: 0.1740 - val_auc: 0.9889 - val_categorical_accuracy: 0.9838 - val_f1: 0.9838 - _timestamp: 1658399891.0000 - _runtime: 162.0000
Epoch 3/10
45/45 [==============================] - 75s 2s/step - loss: 0.4178 - auc: 0.9820 - categorical_accuracy: 0.9681 - f1: 0.9678 - val_loss: 0.2240 - val_auc: 0.9850 - val_categorical_accuracy: 0.9808 - val_f1: 0.9808 - _timestamp: 1658399968.0000 - _runtime: 239.0000
Epoch 4/10
45/45 [==============================] - 97s 2s/step - loss: 0.3171 - auc: 0.9840 - categorical_accuracy: 0.9658 - f1: 0.9654 - v

INFO:tensorflow:Assets written to: gs://geebucketwater/m2_global_Cnn_S1A_el_sl_as/Models/S1A_el_sl_as_EPOCHS_10/assets


auc,▁▅▇▇▇█████
categorical_accuracy,▁▄▇▆▇▇▇█▇█
epoch,▁▂▃▃▄▅▆▆▇█
f1,▁▄▇▆▇▇▇█▇█
loss,█▁▁▁▁▁▁▁▁▁
val_auc,▁▄▁▅▅▆▅█▆█
val_categorical_accuracy,▁▇▅▆▇▇▇█▆█
val_f1,▁▇▅▆▇▇▇█▆█
val_loss,█▄▅▂▂▂▂▁▂▁
GFLOPS,0.0
auc,0.99406


2  loaded 

S1A_el m2
Epoch 1/10
45/45 [==============================] - 85s 1s/step - loss: 60.5380 - auc: 0.9354 - categorical_accuracy: 0.9184 - f1: 0.9186 - val_loss: 0.4513 - val_auc: 0.9823 - val_categorical_accuracy: 0.9821 - val_f1: 0.9821 - _timestamp: 1658400623.0000 - _runtime: 95.0000
Epoch 2/10
45/45 [==============================] - 62s 1s/step - loss: 3.4383 - auc: 0.9654 - categorical_accuracy: 0.9449 - f1: 0.9450 - val_loss: 0.1377 - val_auc: 0.9886 - val_categorical_accuracy: 0.9847 - val_f1: 0.9847 - _timestamp: 1658400685.0000 - _runtime: 157.0000
Epoch 3/10
45/45 [==============================] - 58s 1s/step - loss: 1.4076 - auc: 0.9695 - categorical_accuracy: 0.9486 - f1: 0.9485 - val_loss: 0.0737 - val_auc: 0.9950 - val_categorical_accuracy: 0.9894 - val_f1: 0.9894 - _timestamp: 1658400743.0000 - _runtime: 215.0000
Epoch 4/10
45/45 [==============================] - 100s 2s/step - loss: 1.4742 - auc: 0.9693 - categorical_accuracy: 0.9504 - f1: 0.9516 - val_los

...

auc,▁▅▇▇██████
categorical_accuracy,▁▄▆▇██████
epoch,▁▂▃▃▄▅▆▆▇█
f1,▁▄▆▇██████
loss,█▁▁▁▁▁▁▁▁▁
val_auc,▁▁▆▇██████
val_categorical_accuracy,▁▁▄▇██████
val_f1,▁▁▄▇██████
val_loss,█▄▂▂▁▁▁▁▁▁
GFLOPS,0.0
auc,0.99965


36  loaded 

L8SR_S1 m2
Epoch 1/10
45/45 [==============================] - 98s 1s/step - loss: 2.4809 - auc: 0.9457 - categorical_accuracy: 0.9328 - f1: 0.9334 - val_loss: 0.1060 - val_auc: 0.9907 - val_categorical_accuracy: 0.9824 - val_f1: 0.9824 - _timestamp: 1658427777.0000 - _runtime: 106.0000
Epoch 2/10
45/45 [==============================] - 61s 1s/step - loss: 0.0427 - auc: 0.9980 - categorical_accuracy: 0.9858 - f1: 0.9860 - val_loss: 0.0479 - val_auc: 0.9958 - val_categorical_accuracy: 0.9907 - val_f1: 0.9907 - _timestamp: 1658427839.0000 - _runtime: 168.0000
Epoch 3/10
45/45 [==============================] - 64s 1s/step - loss: 0.0264 - auc: 0.9989 - categorical_accuracy: 0.9913 - f1: 0.9914 - val_loss: 0.0646 - val_auc: 0.9949 - val_categorical_accuracy: 0.9916 - val_f1: 0.9916 - _timestamp: 1658427905.0000 - _runtime: 234.0000
Epoch 4/10
45/45 [==============================] - 98s 2s/step - loss: 0.0203 - auc: 0.9992 - categorical_accuracy: 0.9938 - f1: 0.9939 - val_lo

INFO:tensorflow:Assets written to: gs://geebucketwater/m2_global_Cnn_L8SR_S1/Models/L8SR_S1_EPOCHS_10/assets


auc,▁█████████
categorical_accuracy,▁▇▇███████
epoch,▁▂▃▃▄▅▆▆▇█
f1,▁▇▇███████
loss,█▁▁▁▁▁▁▁▁▁
val_auc,▁▇▆██▇▇█▇█
val_categorical_accuracy,▁▆▇▇█▇▇█▇▇
val_f1,▁▆▇▇█▇▇█▇▇
val_loss,█▂▃▁▁▂▂▁▂▂
GFLOPS,0.0
auc,0.99967


37  loaded 

L8SR_S1A m2
Epoch 1/10


# Multiview-3


<b> The model </b>: Here we use the Keras implementation of the U-Net model. The modified U-Net model takes in three layers of inputs and each has a structure of UNET and and outputs per-pixel class probability. We will use categorical cross entropy as loss function and softmax as the last layer because we are working on a classification problem.

For each experiment, we will:
- Load the training and evaluation dataset from google cloud bucket into a `tf.data.Dataset`.
- Train the Multiview deep learning with 3 inputs  UNET model for 10 epochs
- Store the trained model in google cloud bucket for future prediction
- Store the losses and metrics using Wandb.ai

<b> Note:</b> The time to train the model could take up to 1-2 hours because we are training 8 UNET model that is not pretrained, but this is within the colab background execution runtime. If the connection cuts off, the experiment is continued where the training stops.

In [ ]:
config = {
    "architecture": "Unet",
    "epochs": 10,
    "batch_size": 16
}

df = pd.DataFrame(columns=['Name', 'Loss', 'val_loss',
                           "auc", "val_auc", "accuracy",
                           "val_accuracy", "f1", "val_f1"])

for i in range(0, len(list(configs_multi_3_global))):
    run = wandb.init(project='kl-121-dissertation', reinit=True, config=config)
    print(i + 1, " loaded \n")
    conf = configs_multi_3_global[list(configs_multi_3_global)[i]]
    print(conf.PROJECT_TITLE, conf.type_)
    preproc = preprocessing.Preprocessor(conf)
    training = preproc.get_training_dataset("Train_in_Thailand/")
    evaluation = preproc.get_eval_dataset("Train_in_Thailand/")
    model.CONFIG = conf
    EPOCHS = 10
    wandb.run.name = "Multiview_3" + conf.PROJECT_TITLE + conf.country
    start = time.time()
    EPO = [i for i in range(1, EPOCHS + 1)]
    model_custom = model.get_model_multiview_3()

    history = model_custom.fit(
        x=training,
        epochs=EPOCHS,
        steps_per_epoch=int(240 * 3 / conf.BATCH_SIZE),
        validation_data=evaluation,
        validation_steps=240 * 2,
        callbacks=[WandbCallback()]
    )
    end = time.time()
    print(f'Time for {EPOCHS} epochs is: ', end - start)
    Model_name = conf.PROJECT_TITLE + "_EPOCHS_" + str(EPOCHS)
    MODEL_DIR = 'gs://' + conf.BUCKET + "/" + \
        conf.FOLDER + "/Models/" + Model_name
    model_custom.save(MODEL_DIR, save_format='tf')
    hist_keys = [*history.history]

    df.loc[i] = [conf.PROJECT_TITLE] + \
        [history.history["loss"][-1]] + [history.history["val_loss"][-1]] + \
        [history.history["auc"][-1]] + [history.history["val_auc"][-1]] + \
        [history.history["categorical_accuracy"][-1]] + \
        [history.history["val_categorical_accuracy"][-1]] + \
        [history.history["f1"][-1]] + [history.history["val_f1"][-1]] \

    # import matplotlib.pyplot as plt
    # fig, axes = plt.subplots(1,4, figsize=(20,5))
    # axes[0].plot(EPO, history.history["loss"])
    # axes[0].plot(EPO, history.history["val_loss"])
    # axes[0].legend(['Loss', 'Val_loss'])
    # axes[1].plot(EPO, history.history["auc"])
    # axes[1].plot(EPO, history.history["val_auc"])
    # axes[1].legend(['AUC', 'Val_AUC'])
    # axes[3].plot(EPO, history.history["categorical_accuracy"])
    # axes[3].plot(EPO, history.history["val_cateogorical_accuracy"])
    # axes[3].legend(['accuracy', 'Val_accuracy'])
    # fig.savefig(f'{conf.PROJECT_TITLE} epoch{EPOCHS}.png')
    wandb.log({'Multiview_data_3': df})
    run.finish()

1  loaded 

L8SR_S1_as3 m3
Epoch 1/10
45/45 [==============================] - 120s 2s/step - loss: 17.8284 - auc: 0.9600 - categorical_accuracy: 0.9530 - f1: 0.9530 - val_loss: 0.6048 - val_auc: 0.9853 - val_categorical_accuracy: 0.9828 - val_f1: 0.9828 - _timestamp: 1658432386.0000 - _runtime: 131.0000
Epoch 2/10
45/45 [==============================] - 72s 2s/step - loss: 0.3631 - auc: 0.9868 - categorical_accuracy: 0.9772 - f1: 0.9774 - val_loss: 0.1953 - val_auc: 0.9866 - val_categorical_accuracy: 0.9804 - val_f1: 0.9804 - _timestamp: 1658432458.0000 - _runtime: 203.0000
Epoch 3/10
45/45 [==============================] - 75s 2s/step - loss: 0.1473 - auc: 0.9913 - categorical_accuracy: 0.9789 - f1: 0.9785 - val_loss: 0.1154 - val_auc: 0.9890 - val_categorical_accuracy: 0.9843 - val_f1: 0.9843 - _timestamp: 1658432533.0000 - _runtime: 278.0000
Epoch 4/10
45/45 [==============================] - 98s 2s/step - loss: 0.1061 - auc: 0.9923 - categorical_accuracy: 0.9814 - f1: 0.9809 - v

INFO:tensorflow:Assets written to: gs://geebucketwater/m3_global_Cnn_L8SR_S1_as3/Models/L8SR_S1_as3_EPOCHS_10/assets


auc,▁▆▇▇▇█████
categorical_accuracy,▁▅▅▆▇█▇███
epoch,▁▂▃▃▄▅▆▆▇█
f1,▁▅▅▆▇█▇███
loss,█▁▁▁▁▁▁▁▁▁
val_auc,▁▂▃▇▇█████
val_categorical_accuracy,▂▁▃▄▆▇▇▇██
val_f1,▂▁▃▄▆▇▇▇██
val_loss,█▃▂▁▁▁▁▁▁▁
GFLOPS,0.0
auc,0.99938


2  loaded 

L8SR_S1_el3 m3
Epoch 1/10
45/45 [==============================] - 108s 2s/step - loss: 28.0172 - auc: 0.9346 - categorical_accuracy: 0.9160 - f1: 0.9168 - val_loss: 0.7517 - val_auc: 0.9821 - val_categorical_accuracy: 0.9821 - val_f1: 0.9821 - _timestamp: 1658433318.0000 - _runtime: 120.0000
Epoch 2/10
45/45 [==============================] - 70s 2s/step - loss: 1.2469 - auc: 0.9842 - categorical_accuracy: 0.9683 - f1: 0.9687 - val_loss: 0.1501 - val_auc: 0.9953 - val_categorical_accuracy: 0.9917 - val_f1: 0.9917 - _timestamp: 1658433387.0000 - _runtime: 189.0000
Epoch 3/10
45/45 [==============================] - 66s 1s/step - loss: 0.5296 - auc: 0.9886 - categorical_accuracy: 0.9821 - f1: 0.9823 - val_loss: 0.3288 - val_auc: 0.9944 - val_categorical_accuracy: 0.9923 - val_f1: 0.9923 - _timestamp: 1658433456.0000 - _runtime: 258.0000
Epoch 4/10
45/45 [==============================] - 99s 2s/step - loss: 0.1224 - auc: 0.9958 - categorical_accuracy: 0.9899 - f1: 0.9898 - v

INFO:tensorflow:Assets written to: gs://geebucketwater/m3_global_Cnn_L8SR_S1_el3/Models/L8SR_S1_el3_EPOCHS_10/assets


...

INFO:tensorflow:Assets written to: gs://geebucketwater/m3_global_Cnn_L8SR_S1A_sl3/Models/L8SR_S1A_sl3_EPOCHS_10/assets


auc,▁█████████
categorical_accuracy,▁▇▇███████
epoch,▁▂▃▃▄▅▆▆▇█
f1,▁▇▇███████
loss,█▁▁▁▁▁▁▁▁▁
val_auc,▁▇▄▅█▅▆▆▆▅
val_categorical_accuracy,▁▇▇▇▇█████
val_f1,▁▇▇▇▇█████
val_loss,█▃▃▂▁▂▁▂▂▂
GFLOPS,0.0
auc,0.99975


8  loaded 

L8SR_S1A_sl_el_as3 m3
Epoch 1/10
45/45 [==============================] - 112s 2s/step - loss: 5.0983 - auc: 0.9651 - categorical_accuracy: 0.9565 - f1: 0.9569 - val_loss: 0.1789 - val_auc: 0.9880 - val_categorical_accuracy: 0.9836 - val_f1: 0.9836 - _timestamp: 1658439059.0000 - _runtime: 124.0000
Epoch 2/10
45/45 [==============================] - 79s 2s/step - loss: 0.5101 - auc: 0.9859 - categorical_accuracy: 0.9791 - f1: 0.9793 - val_loss: 0.1006 - val_auc: 0.9944 - val_categorical_accuracy: 0.9917 - val_f1: 0.9917 - _timestamp: 1658439138.0000 - _runtime: 203.0000
Epoch 3/10
45/45 [==============================] - 76s 2s/step - loss: 0.0559 - auc: 0.9966 - categorical_accuracy: 0.9921 - f1: 0.9923 - val_loss: 0.0772 - val_auc: 0.9950 - val_categorical_accuracy: 0.9920 - val_f1: 0.9920 - _timestamp: 1658439214.0000 - _runtime: 279.0000
Epoch 4/10
45/45 [==============================] - 105s 2s/step - loss: 0.0462 - auc: 0.9972 - categorical_accuracy: 0.9921 - f1: 0.9

INFO:tensorflow:Assets written to: gs://geebucketwater/m3_TH_Cnn_L8SR_S1A_sl_el_as3/Models/L8SR_S1A_sl_el_as3_EPOCHS_10/assets


auc,▁▅▇███████
categorical_accuracy,▁▅▇▇██████
epoch,▁▂▃▃▄▅▆▆▇█
f1,▁▅▇▇██████
loss,█▂▁▁▁▁▁▁▁▁
val_auc,▁▆▆▇▇▇████
val_categorical_accuracy,▁▆▇▇▇█████
val_f1,▁▆▇▇▇█████
val_loss,█▄▃▃▂▁▁▁▁▁
GFLOPS,0.0
auc,0.99956
